In [1]:
import cv2
import numpy as np
import glob
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings     
import pandas as pd
import cv2
import pickle
warnings.filterwarnings('ignore')
faceModule = mediapipe.solutions.face_mesh


In [2]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [3]:
jpg_list = glob.glob("AFLW2000/*.jpg")
mat_list = glob.glob("AFLW2000/*.mat")

In [4]:
faceModule = mediapipe.solutions.face_mesh
img_features = []
all_img_features = []
with faceModule.FaceMesh(static_image_mode=True) as faces:
    for i in range(len(mat_list)):
            
            image = cv2.imread(jpg_list[i])
            results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if results.multi_face_landmarks != None: 

                for face in results.multi_face_landmarks:
                    for landmark in face.landmark:
                        x = landmark.x
                        y = landmark.y
                        shape = image.shape 
                        relative_x = int(x * shape[1])
                        relative_y = int(y * shape[0])
                        img_features.append(x)
                        img_features.append(y)
           
            if len(img_features) > 0:
                img_features[0::2] = (img_features[0::2] - np.mean(img_features[0::2])) / max(img_features[0::2] - np.mean(img_features[0::2]))
                img_features[1::2] = (img_features[1::2] - np.mean(img_features[1::2])) / max(img_features[1::2] - np.mean(img_features[1::2]))
                mat_file = sio.loadmat(mat_list[i])
                pose_para = mat_file["Pose_Para"][0][:3]
                pitch = pose_para[0]
                yaw = pose_para[1]
                roll = pose_para[2]
                img_features.append(pitch)
                img_features.append(yaw)
                img_features.append(roll)
                all_img_features.append(img_features)
                img_features = []


In [5]:
df = pd.DataFrame(all_img_features)
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,929,930,931,932,933,934,935,936,937,938
count,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,...,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000
mean,0.050118,0.357779,0.076832,0.113921,0.045066,0.186762,-0.014477,-0.152543,0.078136,0.033892,...,-0.365268,0.143664,-0.346460,0.557394,-0.442963,0.595101,-0.476475,-0.095366,0.013611,-0.033167
std,0.211043,0.084489,0.319450,0.097210,0.187307,0.070779,0.224243,0.063877,0.330106,0.095496,...,0.053772,0.137146,0.047393,0.180097,0.147906,0.189136,0.159078,0.549292,0.580266,0.649282
min,-0.317269,-0.334646,-0.347379,-0.258413,-0.239069,-0.177171,-0.296759,-0.358660,-0.357359,-0.324889,...,-0.495162,-0.405693,-0.439238,-1.515909,-1.072438,-1.684238,-1.144383,-12.457202,-6.130066,-16.969362
25%,-0.080246,0.316445,-0.127263,0.057871,-0.079143,0.148360,-0.171045,-0.194567,-0.141012,-0.023871,...,-0.392614,0.048115,-0.369598,0.445391,-0.517476,0.485262,-0.557641,-0.246453,-0.256714,-0.160274
50%,0.002515,0.363434,-0.011509,0.118988,-0.005902,0.190487,-0.084479,-0.153113,-0.012336,0.038910,...,-0.373379,0.134648,-0.354470,0.588281,-0.450775,0.627190,-0.481904,-0.123371,0.019403,-0.017463
75%,0.104854,0.411350,0.172563,0.176731,0.105981,0.232178,0.061948,-0.115785,0.187013,0.094297,...,-0.349801,0.227109,-0.335001,0.683192,-0.376676,0.725095,-0.405300,0.028133,0.305791,0.126460
max,0.833079,0.657725,0.976003,0.592302,0.631916,0.473462,0.696864,0.218929,0.983205,0.528712,...,0.425557,0.541027,0.401990,0.851204,0.579290,0.891730,0.627716,15.598820,3.277500,2.166107


In [6]:
X = df.iloc[:,:-3]
y = df.iloc[:,-3:]

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn import linear_model

i will Divide Dataset into train , val , test  

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

let's try at first linear regression and it's familly (lasso and ridge)

In [9]:
linear_reg = LinearRegression()
linear_reg.fit(x_train, y_train)
print("Linear Regression Train Score :", linear_reg.score(x_train, y_train))
print("Linear Regression Validation Score :", linear_reg.score(x_val, y_val))
print("Linear Regression Test Score :", linear_reg.score(x_test, y_test))

Linear Regression Train Score : 0.9846423048213852
Linear Regression Validation Score : -4.525342221291463
Linear Regression Test Score : -1.892008099952785


In [10]:
lasso_reg = linear_model.Lasso(alpha=0.001 ,max_iter=10000000)
lasso_reg.fit(x_train, y_train)
ridge_reg = linear_model.Ridge(alpha=0.001, max_iter=10000000)  
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.001, max_iter=10000000)

In [11]:
print("Train Score")
print("Lasso Regression Train Score: ", lasso_reg.score(x_train, y_train))
print("Ridge Regression Train Score: ", ridge_reg.score(x_train, y_train))
print("-----------------")
print("Validation Score")
print("Lasso Regression Validation Score: ", lasso_reg.score(x_val, y_val))
print("Ridge Regression Validation Score: ", ridge_reg.score(x_val, y_val))
print("-----------------")

print("Test Score")
print("Lasso Regression Test Score: ", lasso_reg.score(x_test, y_test))
print("Ridge Regression Test Score: ", ridge_reg.score(x_test, y_test))


Train Score
Lasso Regression Train Score:  0.42773854772741404
Ridge Regression Train Score:  0.747144088066178
-----------------
Validation Score
Lasso Regression Validation Score:  0.6981371222715502
Ridge Regression Validation Score:  -0.4986461572512675
-----------------
Test Score
Lasso Regression Test Score:  0.7468433511756118
Ridge Regression Test Score:  0.13112764639763086


In [70]:
DecisionTree = DecisionTreeRegressor(max_depth=30, random_state=42, max_features=0.6, min_samples_leaf=0.1, min_samples_split=0.1, splitter='best')
DecisionTree.fit(x_train, y_train)
print("Decision Tree Train Score: ", DecisionTree.score(x_train, y_train))
print("Decision Tree Validation Score: ", DecisionTree.score(x_val, y_val))
print("Decision Tree Test Score: ", DecisionTree.score(x_test, y_test))

Decision Tree Train Score:  0.28826812517183015
Decision Tree Validation Score:  0.41425844789872085
Decision Tree Test Score:  0.3913897997847446


In [71]:
random_forest_reg = RandomForestRegressor(max_depth=5,n_estimators=350)
random_forest_reg = random_forest_reg.fit(x_train, y_train)
print("Random Forest Train Score: ", random_forest_reg.score(x_train, y_train))
print("Random Forest Validation Score: ", random_forest_reg.score(x_val, y_val))
print("Random Forest Test Score: ", random_forest_reg.score(x_test, y_test))

Random Forest Train Score:  0.8534033716585916
Random Forest Validation Score:  0.1625596372932959
Random Forest Test Score:  0.3276620329855655


In [62]:
parameters = {
    'estimator__base_estimator': [DecisionTreeRegressor(max_depth=3)],
    'estimator__n_estimators': [50,80,100],
    'estimator__learning_rate': [0.001,0.01,0.1]
}
# Create the GridSearchCV object
grid_search = GridSearchCV(MultiOutputRegressor(AdaBoostRegressor()), parameters)
grid_search.fit(x_train, y_train)

# Get the best AdaBoostRegressor model
best_ada_boost = grid_search.best_estimator_

In [23]:
adaBoostRegressor = MultiOutputRegressor(AdaBoostRegressor(n_estimators=500, learning_rate=0.001, random_state=42))
adaBoostRegressor.fit(x_train, y_train)
print("AdaBoostRegressor Train Score: ", adaBoostRegressor.score(x_train, y_train))
print("AdaBoostRegressor Validation Score: ", adaBoostRegressor.score(x_val, y_val))
print("AdaBoostRegressor Test Score: ", adaBoostRegressor.score(x_test, y_test))

AdaBoostRegressor Train Score:  0.9027194140006677
AdaBoostRegressor Validation Score:  0.695940953385923
AdaBoostRegressor Test Score:  0.7847148216882159


In [66]:
parameters = {
    'estimator__kernel':['rbf','linear','poly'],
    'estimator__C':[1,10,100],
    'estimator__gamma':[0.1,0.05,0.001],
    'estimator__degree':[1,2,3,4,5,6],
    'estimator__epsilon':[0.1,0.01,0.005],
    }
svr_reg = GridSearchCV(MultiOutputRegressor(SVR()), parameters)
svr_reg.fit(x_train, y_train)
best_svr = svr_reg.best_estimator_

In [15]:
svr_reg = MultiOutputRegressor(SVR(kernel='rbf',C=3,gamma=0.005,degree=3,epsilon=0.005))
svr_reg.fit(x_train, y_train)
print("SVR Train Score: ", svr_reg.score(x_train, y_train))
print("SVR Validation Score: ", svr_reg.score(x_val, y_val))
print("SVR Test Score: ", svr_reg.score(x_test, y_test))

SVR Train Score:  0.44307041589666607
SVR Validation Score:  0.8093023899202065
SVR Test Score:  0.9086181751738863


In [ ]:
xgb_regressor = xgb.XGBRegressor(n_estimators=150, max_depth=3, learning_rate=0.009, random_state=42)
multioutput_regressor = MultiOutputRegressor(xgb_regressor)
xgb_reg_winner = multioutput_regressor.fit(x_train, y_train)
print("XGBRegressor Train Score: ", xgb_reg_winner.score(x_train, y_train))
print("XGBRegressor Validation Score: ", xgb_reg_winner.score(x_val, y_val))
print("XGBRegressor Test Score: ", xgb_reg_winner.score(x_test, y_test))

XGBRegressor Train Score:  0.7752879067012247
XGBRegressor Validation Score:  -2.3581231976174206
XGBRegressor Test Score:  -0.2301165780126344


In [91]:
report = pd.DataFrame({
    'Model': ['Linear Regression', 'Ridge','Lasso','Decision Tree', 'Random Forest', 'AdaBoost', 'SVR', 'XGBoost'],
    "Train Score": [linear_reg.score(x_train, y_train), ridge_reg.score(x_train, y_train), lasso_reg.score(x_train, y_train), DecisionTree.score(x_train, y_train), random_forest_reg.score(x_train, y_train), adaBoostRegressor.score(x_train, y_train), svr_reg.score(x_train, y_train), xgb_reg_winner.score(x_train, y_train)],
    "Validation Score": [linear_reg.score(x_val, y_val), ridge_reg.score(x_val, y_val), lasso_reg.score(x_val, y_val), DecisionTree.score(x_val, y_val), random_forest_reg.score(x_val, y_val), adaBoostRegressor.score(x_val, y_val), svr_reg.score(x_val, y_val), xgb_reg_winner.score(x_val, y_val)],
    "Test Score": [linear_reg.score(x_test, y_test), ridge_reg.score(x_test, y_test), lasso_reg.score(x_test, y_test), DecisionTree.score(x_test, y_test), random_forest_reg.score(x_test, y_test), adaBoostRegressor.score(x_test, y_test), svr_reg.score(x_test, y_test), xgb_reg_winner.score(x_test, y_test)]
})
report

,Model,Train Score,Validation Score,Test Score
0,Linear Regression,0.984642,-4.525342,-1.892008
1,Ridge,0.747144,-0.498646,0.131128
2,Lasso,0.427739,0.698137,0.746843
3,Decision Tree,0.288268,0.414258,0.391390
4,Random Forest,0.853403,0.162560,0.327662
5,AdaBoost,0.425087,0.666200,0.730508
6,SVR,0.443070,0.809302,0.908618
7,XGBoost,0.775288,-2.358123,-0.230117


In [96]:
# get the maximum row
report[report['Test Score'] == report['Test Score'].max()]

,Model,Train Score,Validation Score,Test Score
6,SVR,0.44307,0.809302,0.908618


So it's Clear that SVR Is The Best Model In Test And Validate Set So I will Use This Model

In [99]:
filename = 'ridge_model.sav'
pickle.dump(ridge_reg, open(filename, 'wb'))

filename = 'lasso_model.sav'
pickle.dump(lasso_reg, open(filename, 'wb'))

filename = 'DecisionTree_model.sav'
pickle.dump(DecisionTree, open(filename, 'wb'))

filename = 'RandomForest_model.sav'
pickle.dump(random_forest_reg, open(filename, 'wb'))

filename = 'AdaBoost_model.sav'
pickle.dump(adaBoostRegressor, open(filename, 'wb'))

filename = 'SVR_model.sav'
pickle.dump(svr_reg, open(filename, 'wb'))

filename = 'XGBoost_model.sav'
pickle.dump(xgb_reg_winner, open(filename, 'wb'))

In [3]:
# ridge_regression = pickle.load(open('ridge_model.sav', 'rb'))
# lasso_regression = pickle.load(open('lasso_model.sav', 'rb'))
# decision_tree = joblib.load('DecisionTree_model.sav')
svr = pickle.load(open('SVR_model.sav', 'rb'))
# # adaboost = joblib.load('AdaBoost_model.sav')
# xgboost = joblib.load('XGBoost_model.sav')

In [4]:
def extract_landmarks(image):
    img_features = []
    img_features2 = []
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    img_features.append(x)
                    img_features.append(y)
            if len(img_features) > 0:
                img_features2 = img_features.copy()
                img_features[0::2] = (img_features[0::2] - np.mean(img_features[0::2])) / max(img_features[0::2] - np.mean(img_features[0::2]))
                img_features[1::2] = (img_features[1::2] - np.mean(img_features[1::2])) / max(img_features[1::2] - np.mean(img_features[1::2]))
    return img_features , img_features2

In [5]:
def predict_pose(image, model):
    img_features , img_features2 = extract_landmarks(image)
    if image.shape == (0,) or len(img_features) == 0:
        return None
    img_features = np.array(img_features).reshape(1,-1)
    return model.predict(img_features),img_features2

In [10]:
video_path = 'ibrahim.mp4'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4)) 
out = cv2.VideoWriter('test.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
while True:
    ret, frame = cap.read()
    if not ret:
        break
    pose , img_features  = predict_pose(frame, svr) or (None,None)
    if pose is not None:
        pitch, yaw, roll = pose[0]
        # iwant the postion of arrows to be in the middle of the face so what i shoudld give in tdx and tdy in draw axis function
        draw_axis(frame, pitch, yaw, roll,tdx=img_features[10] * frame_width,tdy=img_features[11] * frame_height,size = 100)
    out.write(frame)
